In [1]:
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification
import torch
import pandas as pd

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = "gpt2"  # You can replace with a specific fine-tuned model name if available
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2ForSequenceClassification.from_pretrained(model_name)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
def gpt2_classify(text):
    
    labels = ['ad hominem', 'ad populum', 'appeal to emotion', 'circular reasoning', 'equivocation', 'fallacy of credibility', 'fallacy of extension', 'fallacy of logic', 'fallacy of relevance', 'false causality', 'false dilemma', 'faulty generalization', 'intentional']
    prompt = f"Classify the following text as one of the following logical fallacies: {labels}. {text}. Return only the label."


    input_ids = tokenizer.encode(prompt, return_tensors="pt", max_length=256)  # Adjust max_length as needed
    with torch.no_grad():
        outputs = model(input_ids)
    return labels[torch.argmax(outputs.logits, dim=1).item()]


In [9]:
df = pd.read_csv('data/edu_test.csv')

df = df[['source_article', 'updated_label']]
accurate = 0

for  index, sample in df.iterrows():
    pred = gpt2_classify(sample['source_article'])
    actual = sample['updated_label']
    if (pred == actual):
        accurate += 1
    if index % 50 == 1:
        print("accuracy so far:", accurate / index)
    
    
accuracy = accurate/len(df)
print(accuracy)

accuracy so far: 0.0
accuracy so far: 0.11764705882352941
accuracy so far: 0.09900990099009901
accuracy so far: 0.07947019867549669
accuracy so far: 0.08955223880597014
accuracy so far: 0.09561752988047809
0.1
